### w2v50をテキストにする

In [3]:
cd ../data/external/

/home/saho/project/wovle/data/external


In [2]:
ls

0         1.mecab               filelist              w2v50_fitToDic.tsv
0.filted  bccwj.60k.pdp.htkdic  general_words_300.pd  w2v50.pd
0.mecab   CJLC-0.1/             gm_sorted.csv         w2v50.tsv
1         csj.pdp.htkdic        w2v200_fitToDic.tsv
1.filted  csj.pdp.htkdic.utf8   w2v200.pd
1gm-0000  dictation-kit-v4.4/   w2v200.txt


In [6]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('w2v50/word2vec.gensim.model')
len(model.wv.vocab)

335476

In [7]:
with open('w2v50.tsv','w') as fh:
    for (k,v) in monitor(model.wv.vocab.items()):
        fh.write(k+"\t"+"\t".join([str(e) for e in model.wv[k]])+"\n")

100%|██████████| 335476/335476 [00:46<00:00, 7176.63it/s]


In [8]:
rm -r w2v50/

In [9]:
ls

0         1.mecab               filelist              w2v50_fitToDic.tsv
0.filted  bccwj.60k.pdp.htkdic  general_words_300.pd  w2v50.pd
0.mecab   CJLC-0.1/             gm_sorted.csv         w2v50.tsv
1         csj.pdp.htkdic        w2v200_fitToDic.tsv
1.filted  csj.pdp.htkdic.utf8   w2v200.pd
1gm-0000  dictation-kit-v4.4/   w2v200.txt


### 辞書（csj.pdp.htkdic）に無いものは削除

In [10]:
!nkf -w ./bccwj.60k.pdp.htkdic > ./bccwj.60k.pdp.htkdic.utf8

In [11]:
!head ./bccwj.60k.pdp.htkdic.utf8

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [12]:
words = set()

with open('bccwj.60k.pdp.htkdic.utf8') as hh:
    for line in monitor(hh):
        words.add(line.split('\t')[0].split('+')[0])

with open('w2v50.tsv') as v50h:
    with open('w2v200.txt') as v200h:
        with open('w2v50_fitToDic.tsv','w') as w50h:
            with open('w2v200_fitToDic.tsv','w') as w200h:
                for line in monitor(v50h):
                    w = line.split('\t')[0]
                    if w in words:
                        w50h.write(w+'\t'+'\t'.join(line.split('\t')[1:]))
                for line in monitor(v200h):
                    w = line.split(' ')[0].replace('[','').replace(']','') # 括弧は無いものとして比較
                    if w in words:
                        w200h.write(w+'\t'+'\t'.join(line.split(' ')[1:]))

64274it [00:00, 333431.08it/s]
335476it [00:04, 71818.47it/s]
1015475it [00:34, 29433.44it/s]


In [13]:
!wc -l ./w2v50_fitToDic.tsv

49116 ./w2v50_fitToDic.tsv


In [14]:
!wc -l ./w2v200_fitToDic.tsv

74211 ./w2v200_fitToDic.tsv


### w2v200の括弧重複の平均を取る

#### dataframeとして読み込み

In [4]:
w2v50 = pd.read_csv("w2v50_fitToDic.tsv",sep="\t",header=None)

In [5]:
w2v200 = pd.read_csv("w2v200_fitToDic.tsv",sep="\t",header=None)

In [6]:
w2v = [w2v50, w2v200]

In [7]:
for w in w2v: display(w.shape)

(49116, 51)

(74211, 201)

In [8]:
w2v[0].to_pickle('w2v50.pd')

In [9]:
w2v[1].to_pickle('w2v200.pd')

#### 重複除外

In [10]:
w = w2v[1]

In [11]:
def disp(*v):
    for e in v: display(e)

In [12]:
disp(
    w.ix[[0],:],
    w.shape)

/home/saho/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,の,-0.991739,0.433873,-1.634327,-2.637119,1.87341,-0.68232,1.400241,2.555166,-2.150506,...,-1.206327,0.912507,-0.449425,-0.685022,-0.067702,-0.439921,-1.823313,-0.582093,-0.289067,-0.953114


(74211, 201)

In [13]:
w.ix[w[0] == "トヨタ"]

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
7932,トヨタ,0.053538,-0.458620,2.269142,2.213828,1.045843,-0.385347,1.634038,2.483560,-1.511059,...,-0.565592,-1.320724,1.17779,-0.255538,1.393715,-0.182194,0.562751,-0.150641,0.373629,0.638203
24154,トヨタ,-0.306716,-1.427696,1.110670,1.346888,-0.350039,-0.451238,1.047367,1.017068,-1.072866,...,-0.431226,-0.514442,1.05925,-0.159481,0.126721,-0.407040,-0.150029,0.352745,0.327678,-0.481530


In [14]:
w = w.groupby([0]).mean()

In [15]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
×,-1.418694,0.345751,-0.162140,-0.047963,0.182164,0.381776,0.232795,0.315053,1.269035,-2.457858,...,-0.554922,0.265661,1.001930,-1.022693,-0.188698,-0.251833,-1.842119,-3.492436,0.384303,0.927722
Δ,-0.844599,0.500375,-0.433498,-0.863186,-0.685131,-0.208649,0.063803,1.021694,0.299519,-0.326941,...,0.101914,-0.068191,0.600853,-0.186434,-0.277834,-0.085344,-1.251228,-1.181830,-1.252247,-0.060531
Θ,-0.146901,-0.095641,-0.023297,-0.016684,-0.133185,0.047843,0.085798,0.360042,0.040978,-0.093064,...,0.115077,-0.086670,0.071753,-0.126756,-0.154439,0.117072,-0.070839,-0.238478,-0.136663,-0.044030
Σ,-0.716665,0.317638,-0.243156,0.223840,-0.414557,-0.229860,0.106075,1.233770,0.345052,-0.171819,...,0.015998,-0.482378,0.635308,-0.419857,-0.548821,0.233397,-0.229196,-0.844216,-0.278005,-0.043053
Φ,-0.667787,0.330872,-0.135130,-0.212402,-0.294963,0.128115,0.028918,0.954349,0.370762,-0.191407,...,-0.051202,-0.249389,0.528560,-0.180732,-0.252481,-0.188647,-0.722049,-0.798829,-0.506518,-0.105510


In [16]:
w.shape

(51564, 200)

In [17]:
w.to_pickle('./w2v200.pd')

In [18]:
w = w2v[0]

In [19]:
w.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,ガリバー,0.091114,-0.294387,0.021036,-0.036468,0.101153,0.058209,0.147456,0.059062,0.146042,...,0.239939,-0.006300,0.099647,0.036027,-0.008679,0.039122,-0.049217,0.203507,0.180088,-0.169165
1,くん,0.060573,-0.231098,0.012265,-0.118655,-0.045415,-0.082472,0.005965,-0.031879,0.066911,...,0.133983,-0.022876,0.006493,0.064899,-0.067395,-0.020905,-0.154779,0.175292,0.191491,-0.281831
2,ゴーゴー,0.076387,-0.237495,-0.021150,-0.105880,0.030181,0.174667,0.054576,-0.078703,0.109202,...,0.157529,0.116068,0.040623,0.034536,-0.133727,0.097389,-0.087429,0.222188,0.100252,-0.260785
3,は,0.206208,0.238194,0.164277,-0.219658,0.163085,-0.229095,0.174790,0.049674,-0.146848,...,-0.105047,-0.001622,0.188861,-0.127376,0.117224,0.107232,0.254831,0.029983,0.024894,0.234945
4,、,0.290248,0.130922,0.004911,-0.145435,0.113007,0.071337,0.103400,-0.020992,0.012000,...,-0.175391,0.142215,-0.007413,-0.110463,0.260629,0.084242,0.328284,0.031567,-0.077668,-0.059615


In [20]:
w = w.groupby([0]).mean()

In [21]:
w.shape

(49116, 50)

In [22]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
×,-0.029738,-0.220040,-0.118882,-0.218144,-0.042078,0.001095,0.010656,0.143577,0.059391,0.178090,...,0.010057,0.259506,-0.156964,0.141145,-0.128397,0.120778,0.023176,-0.173646,-0.258388,-0.137577
Δ,-0.084763,0.060476,-0.208180,-0.032915,-0.070210,0.075549,-0.241700,0.069920,0.119192,-0.040702,...,0.037314,0.198016,-0.073684,-0.092430,0.135085,-0.068829,-0.092230,0.004051,-0.205020,0.138573
Θ,0.020196,-0.070559,-0.016031,-0.095259,-0.210722,0.008944,-0.102367,0.049041,0.136521,0.023940,...,0.191656,0.182939,0.138991,-0.162631,0.235224,0.048330,-0.076787,0.091762,0.008522,0.029727
Σ,-0.050522,-0.086176,-0.253005,-0.064997,-0.165054,0.084604,-0.064205,-0.004006,0.055084,0.024274,...,0.099928,0.214353,0.133800,-0.149642,0.083858,-0.048499,0.077051,0.076478,-0.228309,0.165815
Φ,-0.018619,-0.031717,-0.281221,0.003278,-0.084329,0.154924,-0.192161,0.124151,-0.062872,0.035197,...,0.154229,0.307573,0.028767,-0.094916,0.097365,-0.060083,0.124562,-0.013531,-0.255142,0.135849


In [23]:
w.to_pickle('./w2v50.pd')

In [24]:
w2v = []


In [25]:
w2v.append(pd.read_pickle('w2v50.pd'))
w2v.append(pd.read_pickle('w2v200.pd'))

### 非自立語の削除

In [26]:
for (i,w) in enumerate(w2v):
    with open(str(i),"w") as fh:
        for e in w.index:
            fh.write(e+"\n")

In [27]:
!mecab 0 > 0.mecab
!mecab 1 > 1.mecab

In [28]:
!head 0.mecab

×	記号,一般,*,*,*,*,×,カケル,カケル
EOS
Δ	記号,アルファベット,*,*,*,*,Δ,デルタ,デルタ
EOS
Θ	記号,アルファベット,*,*,*,*,Θ,シータ,シータ
EOS
Σ	名詞,固有名詞,一般,*,*,*,Σ,シグマ,シグマ
EOS
Φ	記号,アルファベット,*,*,*,*,Φ,ファイ,ファイ
EOS


In [29]:
!head 1.mecab

×	記号,一般,*,*,*,*,×,カケル,カケル
EOS
Δ	記号,アルファベット,*,*,*,*,Δ,デルタ,デルタ
EOS
Θ	記号,アルファベット,*,*,*,*,Θ,シータ,シータ
EOS
Σ	名詞,固有名詞,一般,*,*,*,Σ,シグマ,シグマ
EOS
Φ	記号,アルファベット,*,*,*,*,Φ,ファイ,ファイ
EOS


In [30]:
 !cat 0.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 0.filted

In [31]:
 !cat 1.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 1.filted

In [32]:
!wc -l 0.filted

47449 0.filted


In [33]:
!wc -l 1.filted

49818 1.filted


In [34]:
w2v_word = [[],[]]
with open('0.filted') as fh:
    w2v_word[0] = [e.split('\t')[0] for e in fh]
with open('1.filted') as fh:
    w2v_word[1] = [e.split('\t')[0] for e in fh]
        

In [35]:
for w in w2v: display(w.shape)

(49116, 50)

(51564, 200)

In [36]:
w2v[0] = w2v[0][w2v[0].index.isin(w2v_word[0])]

In [37]:
w2v[1] = w2v[1][w2v[1].index.isin(w2v_word[1])]

In [38]:
for w in w2v: display(w.shape)

(46977, 50)

(48776, 200)

In [39]:
w2v[0].to_pickle('./w2v50.pd')
w2v[1].to_pickle('./w2v200.pd')